# Exercise 1 - CIFAR with CNNs

In this exercise, we will use convolutional neural networks to solve the CIFAR10 classification task.

You need to load the data from PyTorch, build the appropriate network and train it.


In [1]:
import torch
import torchvision 
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./cifar_data', train = True , download = True, transform = transform )

test_set = torchvision.datasets.CIFAR10('./cifar_data', train = False, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 4, shuffle = True)

test_loader = train_loader = torch.utils.data.DataLoader(test_set, batch_size = 4, shuffle = False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'dear', 'dog', 'frog', 'horse', 'ship', 'truck')



In [5]:
len(train_set)

50000

In [6]:
import torch.nn as nn
import torch.nn.functional as F
#check demension of train_dataset
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5) #input channel size is three because images have 3 colour channels, output = 6, kernel =5
        self.pool = nn.MaxPool2d(2,2) # kernel side = 2 and stride = 2
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # input size
        self.fc2 = nn.Linear(120,90)
        self.fc3 = nn.Linear(90, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) #the number of batches, input size
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x



In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
neural_net = Net().to(device) #if GPU exists 

In [9]:
import torch.optim as optimizer

loss_func = nn.CrossEntropyLoss() #softmax is already includers
optimizer = optimizer.SGD(neural_net.parameters(), lr = 0.01)

total_steps = len(train_loader)

In [10]:
train_loader

In [11]:
for epoch in range(4):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = neural_net(images)
        loss = loss_func(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Loss: {loss.item():.4f}')


print('done')

Loss: 1.5631
Loss: 1.6027
Loss: 1.4576
Loss: 1.2632
done


Evaluating the model...

In [13]:
#this function ensures backward pass and parameter updates in not used 
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_correct_class = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = neural_net(images)
        index, predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(4):
            label = labels[i]
            pred = predicted[i]

            if(label == pred):
                n_correct_class[label] += 1
            n_class_samples[label] += 1

    accuracy = 100.0 * n_correct/n_samples
    print(f'Accuracy of the network: {accuracy} %')

    for i in range(10):
        accuracy = 100.0 * n_correct_class[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {accuracy} % ' )

       

Accuracy of the network: 47.5 %
Accuracy of plane: 67.9 % 
Accuracy of car: 54.1 % 
Accuracy of bird: 35.1 % 
Accuracy of cat: 33.9 % 
Accuracy of dear: 22.7 % 
Accuracy of dog: 54.9 % 
Accuracy of frog: 36.2 % 
Accuracy of horse: 66.4 % 
Accuracy of ship: 59.7 % 
Accuracy of truck: 44.1 % 


# Exercise 2 - IMDB sentiment analysis dataset

In this exercise, we will use an LSTM neural network to classify whether movie reviews from the IMDB dataset are positive or negative.

You need to load the data from PyTorch, build the appropriate network and train it.
